In [6]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

import evaluate
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset


In [4]:

from datasets import DatasetDict

ner_datasets = DatasetDict.load_from_disk("ner_data")
print("ner_datasets", ner_datasets)
print("查看数据集的一条样本：", ner_datasets["train"][0])
print("ner_datasets features: ", ner_datasets["train"].features)
print("")

label_list = ner_datasets["train"].features["ner_tags"].feature.names

print("label_list: ", label_list)



ner_datasets DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})
查看数据集的一条样本： {'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}
ner_datasets features:  {'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

label_list:  ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


In [9]:
# 数据预处理

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base", cache_dir="chinese-macbert-base")

print("tokenizer 打印第一个样本：",tokenizer(ner_datasets["train"][0]["tokens"], is_split_into_words=True))


def process_function(examples):
    tokenizer_examples = tokenizer(examples["tokens"], max_length=128, truncation=True,is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenizer_examples.word_ids(batch_index=1)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenizer_examples["labels"] = labels
    return tokenizer_examples
    



OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like hfl/chinese-macbert-base is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.